In [ ]:
from initialize import *
initialize()

# Test Energy Management System

In [ ]:
printResult("Total energy in the network is initially zero",
            PowerChain.call(METHODS.GET_TOTAL_KWH),0)

printResult("Total ENT in the network is initially zero",
            PowerChain.call(METHODS.GET_TOTAL_ENT),0)

printResult("user1 has zero ENT tokens",
            PowerChain.call(METHODS.BALANCE_ENT,account=user1), [0, 0])

printResult("There are no storage units in the network yet",
            PowerChain.call(METHODS.GET_STORAGE_UNIT_INFO),[])

printResult("Unit1 fails to produce kWhs and mint ENT tokens as it's not yet a storage unit",
            PowerChain.execute(METHODS.ENERGY_PRODUCED,toAddress(unit2.address),123,account=unit1) != "",True)

printResult("Unit1 fails to consume kWhs and burn ENT tokens  as it's not yet a storage unit",
            PowerChain.execute(METHODS.ENERGY_CONSUMED,toAddress(unit2.address),321,account=unit1) != "", True)

PowerChain.execute(METHODS.ADD_STORAGE_UNIT,toAddress(unit1.address),toAddress(user1.address),account=admin)
printResult("Admin makes unit1 a storage unit with owner user1",
            any(u[1] == unit1.address for u in PowerChain.call(METHODS.GET_STORAGE_UNIT_INFO)), True)

PowerChain.execute(METHODS.ADD_VOTER,toAddress(user2.address),account=admin)
printResult("Admin makes user2 a voter",
            PowerChain.call(METHODS.IS_VOTER,account=user2),True)

PowerChain.execute(METHODS.ADD_STORAGE_UNIT,toAddress(unit2.address),toAddress(user3.address),account=admin)
printResult("Admin votes to make unit2 a storage unit with owner user3. user2 has to agree.",
            any(u[1] == unit2.address for u in PowerChain.call(METHODS.GET_STORAGE_UNIT_INFO)), False)

PowerChain.execute(METHODS.ADD_STORAGE_UNIT,toAddress(unit2.address),toAddress(user3.address),account=user2)
storageUnits = PowerChain.call(METHODS.GET_STORAGE_UNIT_INFO)
printResult("User2 votes to make unit2 a storage unit with owner user3. It's now a storage unit",
            any(u[1] == unit1.address for u in storageUnits) and any(u[1] == unit2.address for u in storageUnits), True)

PowerChain.execute(METHODS.REMOVE_VOTER,
                   toAddress(user2.address),account=user2)
PowerChain.execute(METHODS.REMOVE_VOTER,
                   toAddress(user2.address),account=admin)
printResult("User2 and Admin votes to remove user2 from voters and is no longer a voter",
            not PowerChain.call(METHODS.IS_VOTER,account=user2), True)

printResult("User2 produces 10 kWh in unit1 and unit1 reports the production",
            PowerChain.execute(METHODS.ENERGY_PRODUCED,toAddress(user2.address),10000,account=unit1), "")

printResult("Network has 10 kWh in total",
            PowerChain.call(METHODS.GET_TOTAL_KWH), 10000)

printResult("User2 has now 8 ENT (20% is the storage provider fee)",
            PowerChain.call(METHODS.BALANCE_ENT,account=user2), [toInternalNumber(8), 0])

printResult("User1 (owner of unit1 storage unit) has now 2 ENT",
            PowerChain.call(METHODS.BALANCE_ENT,account=user1), [toInternalNumber(2), 0])

printResult("Network has 10 ENT in total",
            PowerChain.call(METHODS.GET_TOTAL_ENT,account=user1), toInternalNumber(10))

printResult("User1 starts a consumption session worth 2 ENT with unit1",
            "session started" in PowerChain.execute(METHODS.START_CONSUMPTION_SESSION,toAddress(unit1.address),toInternalNumber(2),account=user1),True)

printResult("User1 has now one active consumption sessions",
            len(PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user1)), 1)

printResult("Unit1 checks consumption session and has 2kWh",
            PowerChain.call(METHODS.GET_CONSUMPTION_SESSION_ENERGY,toAddress(user1.address),account=unit1),2000)

printResult("User1 has now 2 ENT locked",
            PowerChain.call(METHODS.BALANCE_ENT,account=user1),[0,toInternalNumber(2)])

printResult("User1 consumes 1kWh from the consumption session and unit1 reports it",
            PowerChain.execute(METHODS.ENERGY_CONSUMED,toAddress(user1.address),1000,account=unit1),"")

printResult("User1 has now 1 locked ENT",
            PowerChain.call(METHODS.BALANCE_ENT,account=user1),[0,toInternalNumber(1)])

printResult("User1 checks consumption session and has now 1 kWh",
            PowerChain.call(METHODS.GET_CONSUMPTION_SESSION_ENERGY,toAddress(unit1.address),account=user1), 1000)

printResult("User1 consumes 1kWh from the consumption session and unit1 reports it",
            PowerChain.execute(METHODS.ENERGY_CONSUMED,toAddress(user1.address),1000,account=unit1), "")

printResult("User1 has no longer any active consumption sessions",
            PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user1), [])

printResult("Unit1 has now 8 kWh",
            PowerChain.call(METHODS.GET_STORAGE_UNIT_ENERGY,toAddress(unit1.address)),8000)

printResult("Total network energy is 8 kWh",
            PowerChain.call(METHODS.GET_TOTAL_KWH),8000)

printResult("User3 produces 2 kWh to unit2 and unit2 reports the production",
            PowerChain.execute(METHODS.ENERGY_PRODUCED,toAddress(user3.address),2000,account=unit2), "" )

printResult("User3 has now 2 ENT",
            PowerChain.call(METHODS.BALANCE_ENT,account=user3),[toInternalNumber(2),0])

printResult("User3 start a consumption session with unit2 worth of 3 ENT but fails because it has 2",
            "session started" not in PowerChain.execute(METHODS.START_CONSUMPTION_SESSION,toAddress(unit2.address),toInternalNumber(3),account=user3),True)

printResult("There is no consumption session for user3",
            PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user3), [])

printResult("User3 transfers 2 ENT to user2",
            PowerChain.execute(METHODS.TRANSFER_ENT,toAddress(user2.address),toInternalNumber(2),account=user3), "")

printResult("User2 has now 10 ENT",
            PowerChain.call(METHODS.BALANCE_ENT,account=user2),[toInternalNumber(10),0])

printResult("User2 start a consumption session with unit2 worth of 3 ENT but unit2 don't have enough energy",
            "session started" not in PowerChain.execute(METHODS.START_CONSUMPTION_SESSION,toAddress(unit2.address),toInternalNumber(3),account=user2),True)

printResult("There is no consumption session for user2",
            PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user2), [])

printResult("User2 start a consumption session with unit2 worth of 2 ENT",
            "session started" in PowerChain.execute(METHODS.START_CONSUMPTION_SESSION,toAddress(unit2.address),toInternalNumber(2),account=user2),True)

printResult("There is one consumption session for user2",
            len(PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user2)), 1)

printResult("User2 start a consumption session with unit1 worth of 8 ENT",
            "session started" in PowerChain.execute(METHODS.START_CONSUMPTION_SESSION,toAddress(unit1.address),toInternalNumber(8),account=user2), True)

printResult("There are two consumption sessions for user2",
            len(PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user2)), 2)

printResult("User2 consumes 2kWh from the consumption session with unit2 and unit2 reports it",
            PowerChain.execute(METHODS.ENERGY_CONSUMED,toAddress(user2.address),2000,account=unit2) , "")

printResult("There should be one consumption session now for user2",
            len(PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user2)) , 1)

PowerChain.execute(METHODS.REMOVE_STORAGE_UNIT,toAddress(unit2.address),account=admin)
printResult("Admin votes to remove unit2 from storage units and it's no longer a storage unit",
            any(u[1] == unit2.address for u in PowerChain.call(METHODS.GET_STORAGE_UNIT_INFO)),False)

printResult("User1 consumes 9kWh (over consumption, 1kwh more) from session with unit1 and unit1 reports it",
            PowerChain.execute(METHODS.ENERGY_CONSUMED,toAddress(user2.address),9000,account=unit1) , "")

printResult("There should be no consumption session now for user2",
            PowerChain.call(METHODS.GET_CONSUMPTION_SESSIONS,account=user2), [])

PowerChain.execute(METHODS.REMOVE_STORAGE_UNIT,toAddress(unit1.address),account=admin)
printResult("Admin votes to remove unit1 from storage units and it's no longer a storage unit",
            any(u[1] == unit1.address for u in PowerChain.call(METHODS.GET_STORAGE_UNIT_INFO)),False)